# Demo

## Acquiring codebase

In [1]:
!git clone https://github.com/ArtemShamro/speech-separation

Cloning into 'speech-separation'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (440/440), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 782 (delta 234), reused 266 (delta 129), pack-reused 342 (from 1)
Receiving objects: 100% (782/782), 49.42 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (402/402), done.


## Setting up environment

Обучение лучшего чекпоинта проводилось в окружении с питоном 3.11, на котором установка библиотек ниже не приводит к ошибкам. Однако в коллабе возникают конфликты, которые, впрочем, не фэйлят запуск демонстрации.

In [2]:
%cd speech-separation
!pip install -r requirements.txt
%cd ..

/content/speech-separation
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of numba to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 143.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/content


## Downloading data

In [1]:
import requests
from tqdm import tqdm


def get_direct_download_link(public_link: str) -> str:
    api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download"
    params = {"public_key": public_link}
    response = requests.get(api_url, params=params)
    response.raise_for_status()
    href = response.json().get("href")
    return href


def download_file(url: str, dest_path: str):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))
        with open(dest_path, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest_path) as bar:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                bar.update(len(chunk))

In [2]:
!mkdir speech-separation/data
%cd speech-separation/data

public_link = "https://disk.yandex.ru/d/9k_k6G6a03GURg"
direct_link = get_direct_download_link(public_link)
download_file(direct_link, "dataset.zip")

!unzip dataset.zip
%cd ../..

/content/speech-separation/data


dataset.zip: 100%|██████████| 10.2G/10.2G [07:53<00:00, 21.5MB/s]


Streaming output truncated to the last 5000 lines.
  inflating: dla_dataset/mouths/00238926265.npz  
  inflating: dla_dataset/mouths/00598078100.npz  
  inflating: dla_dataset/mouths/00930801363.npz  
  inflating: dla_dataset/mouths/00699590332.npz  
  inflating: dla_dataset/mouths/00807908112.npz  
  inflating: dla_dataset/mouths/00759804991.npz  
  inflating: dla_dataset/mouths/00593302950.npz  
  inflating: dla_dataset/mouths/00290017507.npz  
  inflating: dla_dataset/mouths/00687403682.npz  
  inflating: dla_dataset/mouths/00184570189.npz  
  inflating: dla_dataset/mouths/00093718036.npz  
  inflating: dla_dataset/mouths/00256102660.npz  
  inflating: dla_dataset/mouths/00998886260.npz  
  inflating: dla_dataset/mouths/00286814065.npz  
  inflating: dla_dataset/mouths/00336540270.npz  
  inflating: dla_dataset/mouths/00049245783.npz  
  inflating: dla_dataset/mouths/00393294484.npz  
  inflating: dla_dataset/mouths/00706426002.npz  
  inflating: dla_dataset/mouths/00390754504.npz  

## Inference val set

Результаты прогонов будут лежать по пути `speech-separation/data/saved/inferencer_results`

In [6]:
%cd speech-separation/
!HYDRA_FULL_ERROR=1 python inference.py \
    --config-name=inference \
    model=dttnet_vision \
    model.g=64 \
    datasets=val \
    datasets.val.limit=320 \
    dataloader.batch_size=16 \
    model.fusion_method="cross_attention_positional" \
    model_loader.from_pretrained="ZenMan67/dla_ss@main"
%cd ..

/content/speech-separation
Loading checkpoint from src/model/VideoEncoders/checkpoints/lrw_resnet18_dctcn_video.pth
[2025-11-24 20:47:33,856][__main__][INFO] - Loading pretrained model from Hugging Face: ZenMan67/dla_ss@main
[2025-11-24 20:47:33,856][__main__][INFO] - Downloading model from Hugging Face Hub: ZenMan67/dla_ss@main (model.pth)
model.pth: 100% 2.48G/2.48G [00:38<00:00, 63.8MB/s]
[2025-11-24 20:48:14,959][__main__][INFO] - Model weights successfully loaded from Hugging Face: ZenMan67/dla_ss@main
[2025-11-24 20:48:14,961][__main__][INFO] - DTTNetVisionModel(
  (video_encoder): VideoEncoder(
    (trunk): ResNet(
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): Swish()
          (relu2): Swish()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), 

## Custom dataset inference

In [7]:
!mkdir speech-separation/data
%cd speech-separation/data

public_link = "https://disk.yandex.com.am/d/lSNdg7-0I1nQkg"
direct_link = get_direct_download_link(public_link)
download_file(direct_link, "mini_dataset.zip")

!unzip mini_dataset.zip
%cd ../..

mkdir: cannot create directory ‘speech-separation/data’: File exists
/content/speech-separation/data


mini_dataset.zip: 100%|██████████| 17.7M/17.7M [00:02<00:00, 8.64MB/s]


Archive:  mini_dataset.zip
replace mini_dla_dataset/audio/mix/00847408841_00439193200.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: mini_dla_dataset/audio/mix/00847408841_00439193200.wav  
  inflating: mini_dla_dataset/audio/mix/00229278733_00447720642.wav  
  inflating: mini_dla_dataset/audio/mix/00026192973_00796376718.wav  
  inflating: mini_dla_dataset/audio/mix/00418353100_00046006024.wav  
  inflating: mini_dla_dataset/audio/mix/00370444191_00325751697.wav  
  inflating: mini_dla_dataset/audio/mix/00349772207_00709895844.wav  
  inflating: mini_dla_dataset/audio/mix/00242660148_00283665620.wav  
  inflating: mini_dla_dataset/audio/mix/00209872203_00439193200.wav  
  inflating: mini_dla_dataset/audio/mix/00906621698_00983807125.wav  
  inflating: mini_dla_dataset/audio/mix/00751903430_00953946353.wav  
  inflating: mini_dla_dataset/audio/mix/00188681592_00260447123.wav  
  inflating: mini_dla_dataset/audio/mix/00796376718_00533993563.wav  
  inflating: mini_dla_dataset

In [8]:
%cd speech-separation/
!HYDRA_FULL_ERROR=1 python inference.py \
    --config-name=inference \
    model=dttnet_vision \
    datasets=val \
    datasets.val.data_path="data/mini_dla_dataset/audio" \
    datasets.val.mouth_path="data/mini_dla_dataset/mouths" \
    datasets.val.dataset_name="mini_dla_dataset" \
    metrics.inference=[] \
    dataloader.batch_size=16 \
    model.fusion_method="cross_attention_positional" \
    model_loader.from_pretrained="ZenMan67/dla_ss@main" \
    inferencer.save_path="mini_inferencer_results"
%cd ..

/content/speech-separation
Loading checkpoint from src/model/VideoEncoders/checkpoints/lrw_resnet18_dctcn_video.pth
[2025-11-24 20:53:54,812][__main__][INFO] - Loading pretrained model from Hugging Face: ZenMan67/dla_ss@main
[2025-11-24 20:53:54,812][__main__][INFO] - Downloading model from Hugging Face Hub: ZenMan67/dla_ss@main (model.pth)
[2025-11-24 20:53:57,975][__main__][WARNING] - Missing keys when loading state_dict: ['fusion.video_proj.0.weight', 'fusion.video_proj.0.bias', 'fusion.q_audio.weight', 'fusion.q_audio.bias', 'fusion.k_video.weight', 'fusion.k_video.bias', 'fusion.v_video.weight', 'fusion.v_video.bias', 'fusion.inp_layernorm.weight', 'fusion.inp_layernorm.bias', 'fusion.attn_layernorm.weight', 'fusion.attn_layernorm.bias', 'fusion.cross_attn.in_proj_weight', 'fusion.cross_attn.in_proj_bias', 'fusion.cross_attn.out_proj.weight', 'fusion.cross_attn.out_proj.bias', 'fusion.cross_ffn.0.weight', 'fusion.cross_ffn.0.bias', 'fusion.cross_ffn.2.weight', 'fusion.cross_ffn.2.